In [1]:
import numpy as np
import pandas as pd
import datatable as dt
import xgboost as xgb
import shap
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, cross_val_score, cross_val_predict
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.metrics import recall_score, accuracy_score
from sklearn.impute import KNNImputer

In [2]:
random_state = 7
cv_method = StratifiedKFold(n_splits=5)

# Training of tsfresh-selected Training Max 2 years and eGFR 4 times

In [3]:
df = dt.fread('/home/jupyter-dchristiadi85/PhD Project 1/data/train_selected_2_4.gz')
df = df.to_pandas()
df.shape

(5958, 23639)

In [108]:
df['aki_1'].fillna(value=0, inplace=True)
df['aki_2'].fillna(value=0, inplace=True)
df['aki_3'].fillna(value=0, inplace=True)

In [ ]:
df.select_dtypes(include='object').columns

In [29]:
df['gn'].value_counts()

no                 5491
igan                190
anca                 85
lupus nephritis      45
membranous           37
unspecific           31
fsgs                 28
mcd                  24
mp/mcgn              19
fgn/itg               6
pign                  2
Name: gn, dtype: int64

In [30]:
df.loc[df.gn == 'pign', 'gn'] = 'unspecific'
df.loc[df.gn == 'fgn/itg', 'gn'] = 'unspecific'

In [31]:
percent_missing = df.isnull().sum()*100 / len(df)
missing_df = pd.DataFrame({'column_name': df.columns, 'percent_miss': percent_missing})

In [32]:
missing_df.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
"crp__fft_coefficient__attr_""""""""""""""""imag""""""""""""""""__coeff_98","crp__fft_coefficient__attr_""""""""""""""""imag""""""""""""""...",100.0
"crp__fft_coefficient__attr_""""""""""""""""real""""""""""""""""__coeff_75","crp__fft_coefficient__attr_""""""""""""""""real""""""""""""""...",100.0
creatinine__query_similarity_count__query_None__threshold_0.0,creatinine__query_similarity_count__query_None...,100.0
mchc__query_similarity_count__query_None__threshold_0.0,mchc__query_similarity_count__query_None__thre...,100.0
"crp__fft_coefficient__attr_""""""""""""""""real""""""""""""""""__coeff_73","crp__fft_coefficient__attr_""""""""""""""""real""""""""""""""...",100.0
...,...,...
htn,htn,0.0
aki_3,aki_3,0.0
aki_2,aki_2,0.0
aki_1,aki_1,0.0


### Number of patients reaching ESKD within training period 

In [33]:
df['cat2'].value_counts()

non_eskd    4614
eskd        1344
Name: cat2, dtype: int64

In [34]:
df.set_index('id', inplace=True)

In [3]:
def to_category(df):
    cols = df.select_dtypes(include='object').columns
    for col in cols:
        df[col] = df[col].astype('category')
    return df

def drop_missing(df):
    threshold = len(df)*0.6
    df.dropna(axis=1, thresh=threshold, inplace=True)
    return df

def copy_df(df):
    return df.copy()

In [37]:
df_cleaned = (df.pipe(copy_df).pipe(drop_missing))
df_cleaned.shape

(5958, 6564)

In [38]:
df_rm_intrain = df_cleaned.query("cat2 == 'non_eskd'")
df_rm_intrain.shape

(4614, 6564)

In [4]:
df_rm_intrain = dt.fread('/home/jupyter-dchristiadi85/PhD Project 1/data/cleaned60_selected_2_4.gz')
df_rm_intrain = df_rm_intrain.to_pandas()
df_rm_intrain.set_index('id', inplace=True)
df_rm_intrain.shape

(4614, 6564)

In [5]:
df_rm_intrain = (df_rm_intrain.pipe(copy_df).pipe(to_category))
df_rm_intrain.shape

(4614, 6564)

### Number of patients reaching ESKD in at 1y post-training 

In [6]:
df_rm_intrain['cat3'].value_counts()

non_eskd    4491
eskd         123
Name: cat3, dtype: int64

### Number of patients reaching ESKD in 2y post-training

In [7]:
df_rm_intrain['cat4'].value_counts()

non_eskd    4388
eskd         226
Name: cat4, dtype: int64

### Number of patients reaching ESKD in 5y post-training

In [10]:
df_rm_intrain['cat7'].value_counts()

non_eskd    4165
eskd         449
Name: cat7, dtype: int64

In [13]:
percent_missing = df_rm_intrain.isnull().sum()*100 / len(df_rm_intrain)
missing_df_rm_intrain = pd.DataFrame({'column_name': df_rm_intrain.columns, 'percent_miss': percent_missing})

In [14]:
missing_df_rm_intrain.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
calcium__autocorrelation__lag_2,calcium__autocorrelation__lag_2,46.142176
calcium__permutation_entropy__dimension_3__tau_1,calcium__permutation_entropy__dimension_3__tau_1,46.098830
"calcium__cwt_coefficients__coeff_2__w_20__widths_(2, 5, 10, 20)",calcium__cwt_coefficients__coeff_2__w_20__widt...,46.098830
"calcium__cwt_coefficients__coeff_2__w_10__widths_(2, 5, 10, 20)",calcium__cwt_coefficients__coeff_2__w_10__widt...,46.098830
calcium__mean_second_derivative_central,calcium__mean_second_derivative_central,46.098830
...,...,...
dkd,dkd,0.000000
htn,htn,0.000000
aki_3,aki_3,0.000000
aki_2,aki_2,0.000000


## Features and Target Input for Training 

In [6]:
dropped_cols = list(df_rm_intrain.columns[-18:])
dropped_cols.remove('age.init')

In [7]:
X = df_rm_intrain.drop(dropped_cols,axis=1).copy()
y = df_rm_intrain['cat7']
X.shape, y.shape

((4614, 6547), (4614,))

In [8]:
y_mapped = y.map({'non_eskd':'no', 'eskd':'yes'})
y_mapped

id
3916     no
3918     no
3921     no
3924     no
3930     no
         ..
23775    no
23780    no
23807    no
23823    no
23864    no
Name: cat7, Length: 4614, dtype: category
Categories (2, object): ['yes', 'no']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, stratify=y_mapped, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((3229, 6547), (1385, 6547), (3229,), (1385,))

In [10]:
lab_encode = LabelEncoder()
y_train = lab_encode.fit_transform(y_train)
y_test = lab_encode.transform(y_test)
y_train.shape, y_test.shape

((3229,), (1385,))

In [11]:
unique_values, counts = np.unique(y_train, return_counts=True)
ratio = dict(zip(unique_values, counts))
ratio

{0: 2915, 1: 314}

In [12]:
imbalance_ratio = ratio[0] / ratio[1]
imbalance_ratio

9.28343949044586

In [13]:
category_cols = list(X_train.select_dtypes(include='category').columns)
numeric_cols = list(X_train.select_dtypes(include=['int', 'float']).columns)
one_hot = OneHotEncoder(sparse=True)
num_imputer = KNNImputer(n_neighbors=5)
len(category_cols), len(numeric_cols)

(4, 6543)

In [14]:
preprocess = ColumnTransformer([('num_imputing', num_imputer, numeric_cols), ('cat_encoder', one_hot, category_cols)], remainder='passthrough')
X_train_processed = preprocess.fit_transform(X_train)
X_train_processed.shape

invalid value encountered in reduce
invalid value encountered in reduce
invalid value encountered in reduce
invalid value encountered in reduce


(3229, 6558)

In [24]:
X_test_processed = preprocess.transform(X_test)
X_test_processed.shape

invalid value encountered in reduce
invalid value encountered in reduce
invalid value encountered in reduce


(1385, 6558)

In [25]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(sampling_strategy=0.5, random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [37]:
scoring = {'acc': 'accuracy', 'prec_macro':'precision_macro', 'rec_macro':'recall_macro', 'roc':'roc_auc'}

In [39]:
cv_scores = cross_validate(estimator=pipeline, X=X_train, y=y_train, scoring=scoring, n_jobs=-1)

In [26]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling',
                 SMOTETomek(random_state=7, sampling_strategy=0.5)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=9.28343949044586, subsample=1,
                               tree_method='exact', use_label_encoder=False,
      

In [27]:
y_pred = pipeline.predict(X_test_processed)

In [28]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[1205,   45],
       [  83,   52]])

In [29]:
roc_auc_score(y_test, y_pred)

0.6745925925925926

In [30]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95      1250
           1       0.54      0.39      0.45       135

    accuracy                           0.91      1385
   macro avg       0.74      0.67      0.70      1385
weighted avg       0.90      0.91      0.90      1385



In [31]:
print(matthews_corrcoef(y_test, y_pred))

0.405820425419609
